# NASA Exoplanet Archive Atmospheric Forward Model Fitting
This notebook is a tutorial for fitting ATMO ([Goyal, J. M. et al., 2019](https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.4503G/abstract)) forward models to [Transmission Spectroscopy data](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=transitspec) from the Exoplanet Archive.

This notebook is meant to require minimal user input, unless you want to provide your own data. Each step will either begin with *(Play)* or **(Input)**. No editing is necessary for the *(Play)* steps, just hover over the cell and press the "Play" button on the left. The **(Input)** steps will tell you what can be changed.

**NOTE:** This has not been tested on all planets with transmission spectroscopy data on the Exoplanet Archive, so your results may vary!

**Estimated Runtime**: 2 minutes

*Authors*: [Kevin Hardegree-Ullman](http://kevinkhu.com), [Sam Grunblatt](skgrunblatt.github.io)

*Last Modified*: November 5, 2024

1.   *(Play)* Install the [SpectRes](https://spectres.readthedocs.io/en/latest/) spectral resampling package.

In [ ]:
!pip install spectres

2.   *(Play)* Import some packages!

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import spectres as spectres

print('Packages successfully imported!')

3.   *(Play)* Read in the [transmission spectroscopy table](https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=transitspec) from the Exoplanet Archive and list the available planets.

In [ ]:
url="https://raw.githubusercontent.com/skgrunblatt/exoplanet_atmospheres/refs/heads/main/transitspec_2024.11.05_13.31.29.csv?dl=1"
tstable=pd.read_csv(url, comment='#', header=0)
tstable.dropna(subset=['plnratror','bandwidth','centralwavelng'], inplace=True, axis=0)
tstable.sort_values('plntname',inplace=True)
tstable.reset_index(drop=True,inplace=True)

print('Here are all the available planets: ',tstable.plntname.unique())

4. **(Input)** Select your favorite planet from the list above.

In [ ]:
planet = 'WASP-121 b' # Edit the name within the quotes. Make sure the exact string matches a planet in the list above.

if tstable['plntname'].str.contains(planet).any():
  print("You have selected planet "+planet+".")
else:
  print("Please check your planet name, it doesn't appear to be in the list above.")

5. *(Play)* Plot the data! Check that things look okay.

In [ ]:
# Extract data from the table above for only the selected planet.
df = tstable.loc[(tstable['plntname'] == planet) & (tstable['centralwavelng'] > 0.3)] #This truncates the data to >0.3 microns, the limit of the models used below.
df = df.dropna(subset=['bandwidth', 'plnratror', 'plnratrorerr1', 'plnratrorerr2'])
df = df.sort_values(by='centralwavelng')
df = df.reset_index(drop=True)

# Make the plot
plt.figure(figsize=(10,8))
plt.title(planet+' Transit Data', fontsize=26)
plt.errorbar(x=df.centralwavelng,y=df.plnratror,xerr=df.bandwidth,yerr=[df.plnratrorerr1,-df.plnratrorerr2],fmt='o',color='red')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.xlabel('Wavelength ($\mathrm{\mu}$m)',fontsize=24)
plt.ylabel('$R_p/R_{\star}$',fontsize=24)

# plt.xscale('log') #uncomment if necessary

6. *(Play)* Download system parameter data from the Exoplanet Archive.

In [ ]:
p2ascii = planet.replace(" ","%20")

url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+*+from+ps+where+pl_name+=+'"+p2ascii+"'+and+default_flag+=+1+&format=csv"

plsyspars=pd.read_csv(url, comment='#', header=0)
plsyspars

7. *(Play)* Take the basic planet and stellar parameters from the table above. Compute planet surface gravity for use below. Feel free to **(Input)** your own values if you have them. (Note: you might have to input your own values for some parameters for certain planets; you can look them up in the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/).)

In [ ]:
#Reference quantities
R_earth = 6371009 #meters
R_jup = 69950000 #meters
R_sun = 695700000 #meters
M_earth = 5.97e24 #kilograms
M_jup = 1.898e27 #kilograms
M_sun = 1.988435e30 #kilograms
G = 6.6743015e-11 #m^3 kg^-1 s^-2

#Planet Parameters
R_guess = plsyspars['pl_rade'][0] * R_earth #Planet radius guess in Earth radii, if that wasn't obvious already
R_err = np.mean([np.abs(plsyspars['pl_radeerr1'][0]),np.abs(plsyspars['pl_radeerr2'][0])]) * R_earth #Planet radius error in Earth radii
print(planet+' Planet Parameters:')
print('R_planet = '+str(round(R_guess/R_earth,2))+' \u00B1 '+str(round(R_err/R_earth,2))+' R_Earth = '+str(round(R_guess/R_jup,2))+' \u00B1 '+str(round(R_err/R_jup,2))+' R_Jupiter')

M_pl = plsyspars['pl_bmasse'][0] * M_earth #Planet mass (or msini) in Earth masses
M_pl_err = np.mean([np.abs(plsyspars['pl_bmasseerr1'][0]),np.abs(plsyspars['pl_bmasseerr2'][0])]) * M_earth #Planet mass error in Earth masses
print('M_planet = '+str(round(M_pl/M_earth,2))+' \u00B1 '+str(round(M_pl_err/M_earth,2))+' M_Earth = '+str(round(M_pl/M_jup,2))+' \u00B1 '+str(round(M_pl_err/M_jup,2))+' M_Jupiter')

grav = G*M_pl/(R_guess**2)
print('Surface Gravity = '+str(round(grav,2))+' m/s^2')

T_guess = 2400#plsyspars['pl_eqt'][0] #Planet equilibrium temperature guess in Kelvin
print('T_eq = '+str(int(T_guess))+' K')
if T_guess < 300 or T_guess > 2600:
  print('*****STOP!***** T_eq is beyond the temperature range of the current model set. The following code will not work. Sorry :(')

#Stellar Host Parameters
Rs = plsyspars['st_rad'][0] * R_sun #Star radius in Sun radii
Rs_err = np.mean([np.abs(plsyspars['st_raderr1'][0]),np.abs(plsyspars['st_raderr2'][0])]) * R_sun #Star radius measurement error in Sun radii
print('\nStellar Host Parameters')
print('R_star = '+str(round(Rs/R_sun,2))+' \u00B1 '+str(round(Rs_err/R_sun,2))+' R_Sun')

T_star = 6400#plsyspars['st_teff'][0] #Star effective temperature in Kelvin
print('T_eff = '+str(int(T_star))+' K')

8. *(Play)* Download [ATMO](https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.4503G/abstract) models closest to the surface gravity and equilibrium temperatures in Step 7.

In [ ]:
#Find closest matching surface gravity and four closest equilibrium temperatures that have models.
modgravarr = [5,10,20,50]
mod = np.abs(modgravarr - grav).tolist()
modgrav = modgravarr[mod.index(min(mod))]
teqlist = [int(np.floor(T_guess/100))*100-100,int(np.floor(T_guess/100))*100,int(np.ceil(T_guess/100))*100,int(np.ceil(T_guess/100))*100+100]
print('Downloading models for g = '+str(modgrav)+' m/s^2 and T_eq = '+str(teqlist[0])+', '+str(teqlist[1])+', '+str(teqlist[2])+', and '+str(teqlist[3])+' K')

#Download ATMO models and combine into a single mega grid.
x=0
for i in teqlist:

  url = 'https://raw.githubusercontent.com/skgrunblatt/exoplanet_atmospheres/main/goyalmodels_local/'+str(modgrav).zfill(2)+'/'+str(i).zfill(4)+'_'+str(modgrav).zfill(2)+'.csv'
  if x == 0:
    model = pd.read_csv(url,header=0)
  else:
    model2 = pd.read_csv(url,header=0)
    model = model.merge(model2,on='wavelength')
  x=1
print('ATMO model download complete!')

9. *(Play)* Find the three ATMO models that fit the data best. Please see [Goyal et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.482.4503G/abstract) for a description of the model parameters.

In [ ]:
#Identify a normalization index for data and models
df.loc[df['plnratror']==df['plnratror'].median()]
normidx = df.loc[df['plnratror']==df['plnratror'].mean()]
try:
  ni = normidx.index.tolist()[0]
except:
  ni = int(len(df)/2)

#Compute (Rp/Rs)^2 and associated errors
df['rprs2'] = (df['plnratror']**2)/(np.array(df['plnratror'][ni])**2)
df['rprs2_err'] = 2*(np.mean([np.abs(df['plnratrorerr1']),np.abs(df['plnratrorerr2'])])/df['plnratror'])*df['rprs2']

#Truncate dataframe to values we care about
dfsm = df[['centralwavelng','bandwidth','rprs2','rprs2_err']].copy()

#Resample high-resolution models to the wavelength scale of the data
columns = list(model)
for i in columns:
  f = spectres.spectres(np.array(df.centralwavelng),np.array(model.wavelength),np.array(model[i]), spec_errs=None)
  dfsm[i] = f/f[ni]

dft = dfsm[['centralwavelng','bandwidth','rprs2','rprs2_err','wavelength']].copy()
dfm = dfsm.drop(['centralwavelng','bandwidth','rprs2','rprs2_err','wavelength'],axis=1)

#Compute the goodness-of-fit statistic (G_k) (Eq. 1 of Cushing et al. (2008)), which is effectively a chi-squared minimization, to identify the best fitting model
fmean = np.mean(dft.rprs2)
dfmn = dfm.div(dfm.mean(axis=0),axis=1)
dfmn = dfmn.multiply(fmean,axis=0)
dfs = dfmn.sub(dft['rprs2'],axis=0)
dfs = dfs.div(dft['rprs2_err'],axis=0)
dfp = np.power(dfs,2)
dfp.loc['Total']=dfp.sum()

#Find the 3 best fitting models!
min1 = dfp.loc['Total'].idxmin()#axis=1)
gk1 = dfp.loc['Total'].min()
dft['best'] = dfmn[min1]
dft['residual'] = (dft.rprs2 - dft.best)/dft.rprs2
dfx = dfp.drop(columns=[min1])
min2 = dfx.loc['Total'].idxmin()#axis=1)
gk2 = dfx.loc['Total'].min()
dfx = dfx.drop(columns=[min2])
min3 = dfx.loc['Total'].idxmin()#axis=1)
gk3 = dfx.loc['Total'].min()

print(planet+' best fitting forward models and associated G_k statistic (smaller indicates a better fit):')
print('(1) T_eq = '+min1[0:4].lstrip('0')+' K, g = '+min1[5:7].lstrip('0')+' m/s^2, log(metallicity_atmo) = '+min1[8:12]+', C/O = '+min1[13:17]+', Haze = '+min1[18:22].lstrip('0')+', Cloud = '+min1[23:27]+', G_k = '+str(round(gk1,2)))
print('(2) T_eq = '+min2[0:4].lstrip('0')+' K, g = '+min2[5:7].lstrip('0')+' m/s^2, log(metallicity_atmo) = '+min2[8:12]+', C/O = '+min2[13:17]+', Haze = '+min2[18:22].lstrip('0')+', Cloud = '+min2[23:27]+', G_k = '+str(round(gk2,2)))
print('(3) T_eq = '+min3[0:4].lstrip('0')+' K, g = '+min3[5:7].lstrip('0')+' m/s^2, log(metallicity_atmo) = '+min3[8:12]+', C/O = '+min3[13:17]+', Haze = '+min3[18:22].lstrip('0')+', Cloud = '+min3[23:27]+', G_k = '+str(round(gk3,2)))

10. *(Play)* Plot your data and the 3 best fit forward models. This will save a file called *planet_name*_best_forward_models.png in the folder to the left.

In [ ]:
#Change the spacing between the three models in the plot below
spacer = 0.2

plt.figure(figsize=(18,16))
plt.xticks(fontsize=24)
plt.yticks(fontsize=24)
plt.title(planet+' Best Fit Forward Models', fontsize=26)
plt.xlabel('Wavelength ($\mathrm{\mu}$m)',fontsize=24)
plt.ylabel('Transit Depth + offset',fontsize=24)

index = abs(model['wavelength'] - df.centralwavelng[ni]).idxmin()
# print(index)
plt.plot(model.wavelength,model[min1]/model[min1][index], lw=3,
         label='$T_{eq}$ = '+min1[0:4].lstrip('0')+' K, $g$ = '
         +min1[5:7].lstrip('0')+' m s$^{-2}$, [Metallicity]$_{atmo}$ = '
         +min1[8:12]+', C/O = '+min1[13:17]+', Haze = '
         +min1[18:22].lstrip('0')+', Cloud = '+min1[23:27])
plt.errorbar(x=dfsm.centralwavelng,y=dfsm.rprs2,xerr=dfsm.bandwidth,yerr=dfsm.rprs2_err,fmt='o',color='black', label=planet+' Transit Data')

plt.plot(model.wavelength,model[min2]/model[min2][index]-spacer, lw=3,
         label='$T_{eq}$ = '+min2[0:4].lstrip('0')+' K, $g$ = '
         +min2[5:7].lstrip('0')+' m s$^{-2}$, [Metallicity]$_{atmo}$ = '
         +min2[8:12]+', C/O = '+min2[13:17]+', Haze = '
         +min2[18:22].lstrip('0')+', Cloud = '+min2[23:27])
plt.errorbar(x=dfsm.centralwavelng,y=dfsm.rprs2-spacer,xerr=dfsm.bandwidth,yerr=dfsm.rprs2_err,fmt='o',color='black')

plt.plot(model.wavelength,model[min3]/model[min3][index]-spacer*2, lw=3,
         label='$T_{eq}$ = '+min3[0:4].lstrip('0')+' K, $g$ = '
         +min3[5:7].lstrip('0')+' m s$^{-2}$, [Metallicity]$_{atmo}$ = '
         +min3[8:12]+', C/O = '+min3[13:17]+', Haze = '
         +min3[18:22].lstrip('0')+', Cloud = '+min3[23:27])
plt.errorbar(x=dfsm.centralwavelng,y=dfsm.rprs2-spacer*2,xerr=dfsm.bandwidth,yerr=dfsm.rprs2_err,fmt='o',color='black')

plt.legend(loc='upper right', fontsize=18)
plt.xlim(np.min(dfsm.centralwavelng)-0.2,np.max(dfsm.centralwavelng)+0.2)
plt.ylim(0.5,1.225)
# plt.xscale('log') #uncomment if necessary
plt.savefig(planet+'_best_forward_models.png')

You have reached the end of this tutorial, congratulations!